In [1]:
import requests 
import time
import os
import json

# Данные с json запросами
datasets_path = "datasets"
json_files = [f for f in os.listdir(datasets_path) if f.endswith(".json")]

for file_name in json_files:
    time_start = time.time()
    print(f"\n=== Обрабатываем файл: {file_name} ===")
    
    file_path = os.path.join(datasets_path, file_name)
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)  # Загружаем данные из файла
    
    print(f"Отправляем данные из {file_name}...")
    
    # Отправляем
    task = requests.post("http://localhost:8001/new", json=data).json()
    task_id = task['taskid']
    print(f"Создана задача: {task_id}")
    
    # Ждем
    while True:
        status = requests.get(f"http://localhost:8001/status?task_id={task_id}").json()['status']
        print(f"Статус: {status}")
        if status == 'DONE':
            break
        time.sleep(2)

    # Получаем результат
    result = requests.get(f"http://localhost:8001/getresult?task_id={task_id}").json()
    print(f"\nРезультат для {file_name}:")
    print(f"DDL команд: {len(result.get('ddl', []))}")
    print(f"Миграций: {len(result.get('migrations', []))}")
    print(f"Запросов: {len(result.get('queries', []))}")
    
    # Показываем первые несколько DDL
    print("\nПримеры DDL:")
    for i, ddl in enumerate(result.get('ddl', [])[:3]):
        print(f"{i+1}. {ddl.get('statement', '')[:100]}...")
    print()
    print(f"Полный ответ:\n {result}")
    print()
    time_end = time.time()
    print(F'Время выполнения {(time_end-time_start)/60:.2f} минут')
    print("-" * 50)
   


=== Обрабатываем файл: flights.json ===
Отправляем данные из flights.json...
Создана задача: ca94d0a4-8b91-44d8-b4bb-89eaec7e2fc1
Статус: DONE

Результат для flights.json:
DDL команд: 2
Миграций: 1
Запросов: 22

Примеры DDL:
1. CREATE SCHEMA IF NOT EXISTS default_catalog.optimized_schema...
2. CREATE TABLE default_catalog.optimized_schema.optimized_table (id INTEGER, data TEXT)...

{'ddl': [{'statement': 'CREATE SCHEMA IF NOT EXISTS default_catalog.optimized_schema'}, {'statement': 'CREATE TABLE default_catalog.optimized_schema.optimized_table (id INTEGER, data TEXT)'}], 'migrations': [{'statement': "INSERT INTO default_catalog.optimized_schema.optimized_table SELECT 1, 'optimized_data'"}], 'queries': [{'queryid': '10ba3c04-0f91-4ef3-a717-c1e0d33b31bc', 'query': 'SELECT * FROM default_catalog.optimized_schema.optimized_table WHERE id = 1', 'runquantity': 795}, {'queryid': '8abd47c0-31cb-4ba0-891f-9bac53bbc909', 'query': 'SELECT * FROM default_catalog.optimized_schema.optimized_table W

In [2]:
import asyncio
import aiohttp
import time
import os
import json

# Данные с json запросами
datasets_path = "datasets"
json_files = [f for f in os.listdir(datasets_path) if f.endswith(".json")]

async def process_file(session, file_name):
    time_start = time.time()
    print(f"\n=== Обрабатываем файл: {file_name} ===")

    file_path = os.path.join(datasets_path, file_name)
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    print(f"Отправляем данные из {file_name}...")

    # Отправляем
    async with session.post("http://localhost:8001/new", json=data) as resp:
        task = await resp.json()
    task_id = task["taskid"]
    print(f"Создана задача: {task_id}")

    # Ждем
    while True:
        async with session.get(f"http://localhost:8001/status?task_id={task_id}") as resp:
            status = (await resp.json())["status"]
        print(f"Статус: {status}")
        if status == "DONE":
            break
        await asyncio.sleep(2)

    # Получаем результат
    async with session.get(f"http://localhost:8001/getresult?task_id={task_id}") as resp:
        result = await resp.json()

    print(f"\nРезультат для {file_name}:")
    print(f"DDL команд: {len(result.get('ddl', []))}")
    print(f"Миграций: {len(result.get('migrations', []))}")
    print(f"Запросов: {len(result.get('queries', []))}")

    print("\nПримеры DDL:")
    for i, ddl in enumerate(result.get("ddl", [])[:3]):
        print(f"{i+1}. {ddl.get('statement', '')[:100]}...")
    print()
    print(f"Полный ответ:\n {result}")
    print()
    time_end = time.time()
    print(f"Время выполнения {(time_end - time_start)//60:.2f} минут")
    print("-" * 50)


async def main():
    connector = aiohttp.TCPConnector(limit=10)
    async with aiohttp.ClientSession(connector=connector) as session:
        tasks = []
        # Берем только первые два файла для примера
        for file_name in json_files[:2]:
            tasks.append(asyncio.create_task(process_file(session, file_name)))

        await asyncio.gather(*tasks)


await main()



=== Обрабатываем файл: flights.json ===
Отправляем данные из flights.json...

=== Обрабатываем файл: questsH.json ===
Отправляем данные из questsH.json...
Создана задача: a060f41f-7a1f-4228-a447-912a1641925d
Создана задача: 7ef8a62f-925f-49bd-bbcd-fb4053a16cff
Статус: DONE
Статус: DONE

Результат для questsH.json:
DDL команд: 2
Миграций: 1
Запросов: 11

Примеры DDL:
1. CREATE SCHEMA IF NOT EXISTS default_catalog.optimized_schema...
2. CREATE TABLE default_catalog.optimized_schema.optimized_table (id INTEGER, data TEXT)...

{'ddl': [{'statement': 'CREATE SCHEMA IF NOT EXISTS default_catalog.optimized_schema'}, {'statement': 'CREATE TABLE default_catalog.optimized_schema.optimized_table (id INTEGER, data TEXT)'}], 'migrations': [{'statement': "INSERT INTO default_catalog.optimized_schema.optimized_table SELECT 1, 'optimized_data'"}], 'queries': [{'queryid': '27a8890e-63d2-4078-a412-00ed39604ffc', 'query': 'SELECT * FROM default_catalog.optimized_schema.optimized_table WHERE id = 1', 'ru